<img src="https://cienciasgeodesicas.ufpr.br/wp-content/uploads/2021/03/cropped-folder-site3.png" alt="Drawing"/>

**Federal University of Parana, Curitiba, Brazil**

Geodetic Science Graduate Programme

---

Authors:
 - Darlan Miranda Nunes | [ORCID- 0000-0003-3557-5341](https://orcid.org/0000-0001-5566-7919)
 - Silvana Philippi Camboim | [ORCID- 0000-0003-3557-5341](https://orcid.org/0000-0003-3557-5341)
---

## Collaborative Toponyms in OpenStreetMap: an open-source framework to investigate the relationship with intrinsic quality parameters

DOI: [https://doi.org/10.1080/15230406.2025.2566807](https://doi.org/10.1080/15230406.2025.2566807)

**Aims**

- To conduct a quantitative assessment of elements within OpenStreetMap (OSM) that have the 'name' attribute filled for potential categories of the Brazilian Authoritative Topographic Map; and

- To investigate the most significant intrinsic quality parameters that contribute to the reliability of toponyms in OSM.


**Brief Overview of the Proposed Methodology**

- Preliminary survey of potential OpenStreetMap (OSM) tags to provide relevant toponym information to categories of interest related to Brazilian Topographic Mapping;

- Execution of a quantitative analysis on collaboratively entered toponyms, utilizing homogeneous grid-based approaches; and

- Assessment of intrinsic quality parameters as indicators of the reliability of toponyms in a scientific context.

---

## Jupyter notebook 01: Retrieving data from OpenStreetMap using OHSOME API and homogeneous grid cells

### Set Output File Info ***(Manual Input Required)*** and Import the libraries

In [1]:
# Set output file info (AMEND MANUALLY)
# All info is in the input file name. Check it if unsure.
# Input files in 'data' folder, subfolder 'input_code1'.

# What is the study area? (Only letters/digits/underscores, in quotes.)
study_area = "amsterdam"

# What is the grid size? (Only letter/digits, in quotes. No underscores!)
grid_size = "100m"

# Are you using the "full" input dataset or the "trimmed" dataset?
# (Check filename if unsure. Just input "full" or "trimmed". No underscores!)
grid_extent = "full"

In [2]:
# Import library and some pre-installed modules
import os
import requests
import json
import pandas as pd
import geopandas as gpd
import time
from ipywidgets import widgets
import folium
from IPython.display import display
from shapely.geometry import shape, mapping
from pyproj import Transformer
from shapely.ops import transform
import copy

### Homogeneous Grid Cells
Statistical Grid (cell size 100 x 100m) produced by Gemeente Amsterdam:

https://api.data.amsterdam.nl/v1/docs/datasets/beheerkaart/cbs_grid.html

#### Import Homogeneous Grid Cells from Google Drive

In [3]:
# Import the statistics grid in GeoJSON format

grid = None

# Function for selecting and loading the GeoJSON file
def select_file(change):
    global grid
    selected_file = change['new']
    
    if selected_file != "Select the GeoJSON file with grid cells:":
        file_path = os.path.join('../data/input_code1/', selected_file)
        try:
            with open(file_path, 'r') as file:
                grid = json.load(file)
            print("File selected with success:", selected_file)
            print("File path:", file_path)
        except FileNotFoundError:
            print("File not found:", selected_file)

# Listing available GeoJSON files
file_list = [f for f in os.listdir('../data/input_code1/') if f.endswith('.geojson')]
options = ["Select the GeoJSON file with grid cells:"] + file_list

# Dropdown to select the GeoJSON file
dropdown = widgets.Dropdown(options=options)
dropdown.observe(select_file, names='value')

# Display the dropdown
display(dropdown)

Dropdown(options=('Select the GeoJSON file with grid cells:', 'Amsterdam_100_grid_trimmed.geojson', 'Amsterdam…

In [4]:
# Count the total number of grid cells in GeoJSON
total_cells = len(grid['features'])
print(f"Total grid cells in GeoJSON: {total_cells}")

Total grid cells in GeoJSON: 1


In [5]:
# CRS transformer: RD New → WGS84
transformer = Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)

def reproject_featurecollection_to_wgs84(fc):
    fc = copy.deepcopy(fc)

    for feature in fc["features"]:
        geom = shape(feature["geometry"])
        geom_wgs = transform(
            lambda x, y: transformer.transform(x, y),
            geom
        )
        feature["geometry"] = mapping(geom_wgs)

    fc["crs"] = {
        "type": "name",
        "properties": {"name": "EPSG:4326"}
    }

    return fc

grid = reproject_featurecollection_to_wgs84(grid)


In [6]:
# Partition the original GeoJSON grid into subsets of up to 4 cells each

# Number of cells per batch
subset_size = 4

# Split the original grid cells into subsets
subsets = [grid['features'][i:i + subset_size] for i in range(0, len(grid['features']), subset_size)]

# Create a new FeatureCollection structure for each subset and add a batch ID ("lot_id")
grid_subsets = []
for index, subset in enumerate(subsets):
    grid_subset = {
        'type': 'FeatureCollection',
        'features': subset,
        'lot_id': f"lot{index + 1}",
        'crs': grid['crs']
    }
    grid_subsets.append(grid_subset)

In [7]:
# Calculate and print the total of subsets created
total_subsets = len(grid_subsets)
print(f"Total subsets created: {total_subsets}")

Total subsets created: 1


#### Visualize the spatial distribution of the homogeneous grid cell

In [8]:
# Visualises the grid cell subsets using Folium
# The CRS used for analysis (EPSG:28992) does not work with Folium.
# Hence, a deep copy of the grid is made and reprojected to EPSG:4326.

# Compute map centroid (EPSG:4326)
geom = shape(grid["features"][0]["geometry"])
centroid_coords = [geom.centroid.y, geom.centroid.x]

print("Map center (lat, lon):", centroid_coords)

# Set name of ID field
if grid.get("name") == "grid_AmsterdamCentrum_100m_reproj":
    id_field = "bkCbsGrid100Asd"
elif grid.get("name") == "grid_AmsterdamCentrum_200m":
    id_field = "id"
else:
    id_field = "id"  # fallback

# Plot function (called by dropdown)
def plot_subset(subset_index):
    subset = grid_subsets[subset_index]

    # Create map
    m = folium.Map(
        location=centroid_coords,
        tiles="OpenStreetMap",
        zoom_start=14
    )

    # Style
    style = {
        "fillColor": "#8C8989",
        "color": "#e31a1c",
        "weight": 2,
        "fillOpacity": 0.6
    }

    # Add grid
    folium.GeoJson(
        subset,
        name=f"Statistical Grid - Lot {subset_index + 1}",
        style_function=lambda x: style,
        tooltip=folium.GeoJsonTooltip(
            fields=[id_field],
            aliases=["Grid Cell ID"]
        )
    ).add_to(m)

    display(m)

# Dropdown widget 
dropdown = widgets.Dropdown(
    options=[(f"Lot {i + 1}", i) for i in range(len(grid_subsets))],
    description="Select a Batch:",
)

widgets.interactive(plot_subset, subset_index=dropdown)

Map center (lat, lon): [52.379265953549506, 4.888598835750396]


interactive(children=(Dropdown(description='Select a Batch:', options=(('Lot 1', 0),), value=0), Output()), _d…

### **OHSOME API**

 - Access to features, attributes and OSM history edits using the OHSOME API (*OpenStreetMap History Data Analytics Platform*)

> - https://docs.ohsome.org/ohsome-api/v1/


In [9]:
# URL of OHSOME API Metadata endpoint
URL = 'https://api.ohsome.org/v1/metadata'

# Request to the OHSOME API
response = requests.get(URL)

if response.status_code != 200:
    print("Request failed")
    print("Status:", response.status_code)
    print("Content-Type:", response.headers.get("Content-Type"))
    print(response.text[:800])
else:
    response_json = response.json()
    response_json

### Retrieving data from OpenStreetMap using OHSOME API and homogeneous grid cells


#### Step 1 (*API Endpoint: Elements Aggregation*): count the number of OSM features (elements) and calculate the proportion of features with the attribute "name" fill in by contributors, for each grid cells:


 - Determine the total number of OSM features for interest tags, grouped by grid cell;

 - Quantify the total number of features with attribute "name" filled in; and

 - Calculate the proportion of features with attribute "name" filled in for each grid cell.

 - Period of data retrieved: 2007-10-08 to 2024-03-10;

In [11]:
# Approach for processing batches of 04 cells from the original grid

# Step 1 (API Endpoint: Elements Aggregation): count the number of OSM features
# (elements) and calculate the proportion of features with "name" attribute filled by contributors, for each grid cells:
# Aggregation method: count
# POST /elements/(aggregation)/groupBy/boundary/groupBy/tag

def post_with_retry(url, data, max_retries=5):
    """
    POST to the given URL with retries for network errors or timeouts.
    Exponential backoff: waits 2,4,8,... seconds between attempts.
    Waits indefinitely for the response (no timeout).
    """
    for attempt in range(1, max_retries + 1):
        try:
            response = requests.post(url, data=data)  # no timeout
            response.raise_for_status()  # raises HTTPError for 4xx/5xx
            return response
        except requests.exceptions.RequestException as e:
            wait = 2 ** attempt
            print(f"Attempt {attempt} failed: {e}. Retrying in {wait}s...")
            time.sleep(wait)
    raise RuntimeError(f"Failed to connect after {max_retries} attempts")

# Start the time counter
start_time = time.time()

# Load a copy of previously created grid_subsets
grid_subset2 = grid_subsets.copy()

# OHSOME API endpoint url
url_tag = "https://api.ohsome.org/v1/elements/count/groupBy/boundary/groupBy/tag"

# OSM tags of interest
tags_de_interesse = {
    'leisure': '*',
    'building': '*',
    'amenity': '*'
}

# Configuring basic parameters
params_base = {
    'time': '2007-10-08/2024-03-10'
}

# List to store the final results
final_results = {}

# Process each batch of grid_subsets
for lot_id, subset in enumerate(grid_subset2, start=1):
    for feature in subset['features']:
        cell_geojson = json.dumps({"type": "FeatureCollection", "features": [feature]})
        cell_id = feature['properties']['id']

        for tag, value in tags_de_interesse.items():
            # 1st: Aggregate the object of each interest tag by grid cell
            params = params_base.copy()
            params.update({
                'bpolys': cell_geojson,
                'filter': f'{tag}={value}',
                'groupByKey': tag,
                'groupByValues': value
            })

            try:
                response = post_with_retry(url_tag, data=params)
                data = response.json()
                total_count = sum(res.get('value', 0) for res in data.get('groupByResult', [])[0].get('result', []))
                feature['properties'][f'{tag}_total_count'] = total_count
            except RuntimeError:
                print(f"Warning: request failed for cell {cell_id}, tag {tag} after multiple attempts")
                total_count = 0

            # 2nd: Count the features with the attribute 'name' filled in
            params['filter'] = f'{tag}={value} and name=*'
            try:
                response = post_with_retry(url_tag, data=params)
                data = response.json()
                name_count = sum(res.get('value', 0) for res in data.get('groupByResult', [])[0].get('result', []))
                feature['properties'][f'{tag}_name_count'] = name_count
                name_ratio_perc = (name_count / total_count) * 100 if total_count > 0 else 0
                feature['properties'][f'{tag}_name_ratio'] = name_ratio_perc
            except RuntimeError:
                print(f"Warning: request failed for cell {cell_id}, tag {tag} after multiple attempts")
                name_count = 0
                
        # Add cell results to final_results
        final_results[cell_id] = feature['properties']

    print(f"{subset['lot_id']} successfully processed!")

# Stop the time counter
end_time = time.time()

# Calculate and display the total execution time
total_time_seconds = end_time - start_time
print(f"Total execution time: {total_time_seconds // 60} minutes and {total_time_seconds % 60} seconds")

lot1 successfully processed!
lot2 successfully processed!
lot3 successfully processed!
lot4 successfully processed!
lot5 successfully processed!
lot6 successfully processed!
lot7 successfully processed!
lot8 successfully processed!
lot9 successfully processed!
lot10 successfully processed!
lot11 successfully processed!
lot12 successfully processed!
lot13 successfully processed!
lot14 successfully processed!
lot15 successfully processed!
lot16 successfully processed!
lot17 successfully processed!
lot18 successfully processed!
lot19 successfully processed!
lot20 successfully processed!


KeyboardInterrupt: 

In [ ]:
# Save or append step results to a GeoJSON file

# Output file using your naming convention
output_filename = f"../data/output_code1/{study_area}_{grid_size}_{grid_extent}_results.geojson"

# Collect current step features into a dict keyed by cell ID
current_features = {}

for subset in grid_subset2:
    for feature in subset['features']:
        cell_id = feature['properties']['id']
        current_features[cell_id] = feature


if os.path.exists(output_filename):
    # Load existing data
    with open(output_filename, 'r', encoding='utf-8') as f:
        existing_fc = json.load(f)

    # Index existing features by cell ID
    existing_features = {
        feat['properties']['id']: feat
        for feat in existing_fc['features']
    }

    # Merge: update properties of existing features
    for cell_id, new_feature in current_features.items():
        if cell_id in existing_features:
            existing_features[cell_id]['properties'].update(
                new_feature['properties']
            )
        else:
            # New cell (should not usually happen, but safe)
            existing_features[cell_id] = new_feature

    # Rebuild FeatureCollection
    merged_fc = {
        'type': 'FeatureCollection',
        'crs': existing_fc.get('crs'),
        'features': list(existing_features.values())
    }

else:
    # First step: just save everything
    merged_fc = {
        'type': 'FeatureCollection',
        'crs': grid_subset2[0]['crs'],
        'features': list(current_features.values())
    }

# Write to disk
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(merged_fc, f, ensure_ascii=False, indent=2)

print(f"Results merged and saved to: {output_filename}")

#### Step 2 (*API Endpoint: Contributions Aggregation*): count the total number of contributions for features with and without the attribute "name" filled in:

- Count the **total number of contributions** to the *interest tags* for the total features in the grid cells, with and without the attribute "name" filled in.

- Period of data retrieved: 2007-10-08 to 2024-03-10.

In [16]:
# Step 2 (API Endpoint: Endpoint Contributions Aggregation): count the total number of
# contributions for features with and without a name attribute filled in.
# Aggregation method: count 
# POST /contributions/count/groupBy/boundary

start_time = time.time()

url_contributions = "https://api.ohsome.org/v1/contributions/count/groupBy/boundary"

tags_de_interesse = {
    'leisure': '*',
    'building': '*',
    'amenity': '*'
}

params_contributions_base = {
    'time': '2021-03-11/2024-03-10'
}

def process_response(response, cell_id):
    data = response.json()
    return sum(result.get('value', 0)
               for result in data.get('groupByResult', [])[0].get('result', []))


for lot_id, subset in enumerate(grid_subset2, start=1):
    for feature in subset['features']:
        cell_geojson = json.dumps({"type": "FeatureCollection", "features": [feature]})
        cell_id = feature['properties']['id']

        for tag in tags_de_interesse:

            # --- All contributions ---
            params_contributions_all = params_contributions_base.copy()
            params_contributions_all.update({
                'bpolys': cell_geojson,
                'filter': f'{tag}=*'
            })

            try:
                response_all = post_with_retry(url_contributions, params_contributions_all)
                contributions_all = process_response(response_all, cell_id)
            except RuntimeError:
                print(f"Skipping cell {cell_id} for tag {tag} (connection failed)")
                contributions_all = 0

            # --- Named contributions ---
            params_contributions_name = params_contributions_base.copy()
            params_contributions_name.update({
                'bpolys': cell_geojson,
                'filter': f'{tag}=* and name=*'
            })

            try:
                response_name = post_with_retry(url_contributions, params_contributions_name)
                contributions_name = process_response(response_name, cell_id)
            except RuntimeError:
                print(f"Skipping name contributions for cell {cell_id}, tag {tag}")
                contributions_name = 0

            feature['properties'][f'{tag}_total_contributions'] = contributions_all
            feature['properties'][f'{tag}_name_contributions'] = contributions_name

    print(f"{subset['lot_id']} successfully processed!")

end_time = time.time()
total_time_seconds = end_time - start_time
print(f"Total execution time: {total_time_seconds // 60} minutes and {total_time_seconds % 60} seconds")

lot1 successfully processed!
Total execution time: 0.0 minutes and 53.59541726112366 seconds


In [17]:
# Save or append step results to a GeoJSON file

# Collect current step features into a dict keyed by cell ID
current_features = {}

for subset in grid_subset2:
    for feature in subset['features']:
        cell_id = feature['properties']['id']
        current_features[cell_id] = feature


if os.path.exists(output_filename):
    # Load existing data
    with open(output_filename, 'r', encoding='utf-8') as f:
        existing_fc = json.load(f)

    # Index existing features by cell ID
    existing_features = {
        feat['properties']['id']: feat
        for feat in existing_fc['features']
    }

    # Merge: update properties of existing features
    for cell_id, new_feature in current_features.items():
        if cell_id in existing_features:
            existing_features[cell_id]['properties'].update(
                new_feature['properties']
            )
        else:
            # New cell (should not usually happen, but safe)
            existing_features[cell_id] = new_feature

    # Rebuild FeatureCollection
    merged_fc = {
        'type': 'FeatureCollection',
        'crs': existing_fc.get('crs'),
        'features': list(existing_features.values())
    }

else:
    # First step: just save everything
    merged_fc = {
        'type': 'FeatureCollection',
        'crs': grid_subset2[0]['crs'],
        'features': list(current_features.values())
    }

# Write to disk
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(merged_fc, f, ensure_ascii=False, indent=2)

print(f"Results merged and saved to: {output_filename}")

Results merged and saved to: ../data/output_code1/Amsterdam_100m_full_results.geojson


#### Step 3 (*API Endpoint: Contributions Aggregation*): Count the number of contributions in the past five years for features with the attribute "name" filled in:

 - Count the number of contributions in the past five years for tags of interest, aggregated by grid cells, with the attribute "name" filled in;

 - Period of data retrieved: 2019-03-09 to 2024-03-10

In [18]:
# Step 3 (API Endpoint: Contributions Aggregation)
# Count the number of contributions in the past five years
# for features with a filled-in name
# POST /contributions/latest/count

start_time = time.time()

url_latest_contributions = "https://api.ohsome.org/v1/contributions/latest/count"

tags_de_interesse = {
    'leisure': '*',
    'building': '*',
    'amenity': '*'
}

params_contributions_base = {
    'time': '2019-03-09/2024-03-10'
}

def process_response(response, cell_id):
    data = response.json()
    latest_result = data.get('result', [])
    return latest_result[-1].get('value', 0) if latest_result else 0


for lot_id, subset in enumerate(grid_subset2, start=1):
    for feature in subset['features']:
        cell_geojson = json.dumps({
            "type": "FeatureCollection",
            "features": [feature]
        })
        cell_id = feature['properties']['id']

        for tag in tags_de_interesse:
            params_latest_contributions = params_contributions_base.copy()
            params_latest_contributions.update({
                'bpolys': cell_geojson,
                'filter': f'{tag}=* and name=*'
            })

            try:
                response = post_with_retry(
                    url_latest_contributions,
                    params_latest_contributions
                )
                latest_contributions_count = process_response(response, cell_id)
            except RuntimeError:
                print(f"Skipping cell {cell_id} for tag {tag} (connection failed)")
                latest_contributions_count = 0

            feature['properties'][f'{tag}_latest5_name_contributions'] = (
                latest_contributions_count
            )

    print(f"{subset['lot_id']} successfully processed!")

end_time = time.time()
total_time_seconds = end_time - start_time
print(
    f"Total execution time: {total_time_seconds // 60} minutes "
    f"and {total_time_seconds % 60} seconds"
)

lot1 successfully processed!
Total execution time: 0.0 minutes and 26.292473793029785 seconds


In [ ]:
# Save or append step results to a GeoJSON file

# Create a FeatureCollection from the current step
current_step_fc = {
    'type': 'FeatureCollection',
    'crs': grid_subset2[0]['crs'],
    'features': []
}

for subset in grid_subset2:
    current_step_fc['features'].extend(subset['features'])

# Check if file already exists
if os.path.exists(output_filename):
    # Load existing features
    with open(output_filename, 'r', encoding='utf-8') as f:
        existing_data = json.load(f)
    # Append current step features
    existing_data['features'].extend(current_step_fc['features'])
    combined_fc = existing_data
else:
    combined_fc = current_step_fc

# Save combined results back to the file
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(combined_fc, f, ensure_ascii=False, indent=2)

print(f"Step results appended and saved to: {output_filename}")

#### Step 4 (*API Endpoint: Contributions Aggregation*): Count the total number of contributions to features with a filled-in name where a tagChange occurred:

- Count the total number of contributions to the tags of interest, aggregated by grid cell, with the attribute name filled in, considering the type of contribution (contributionType) tag change ('tagChange').

  - *contributionType available: ‘creation’, ‘deletion’, ‘tagChange’, ‘geometryChange’ ou uma combinação destes*

- Period of data retrieved: 2007-10-08 to 2024-03-10.

In [12]:
# Step 4 (API Endpoint: Contributions Aggregation)
# Count total contributions with tagChange
# for features with a filled-in name
# POST /contributions/count/groupBy/boundary

start_time = time.time()

url_contributions = "https://api.ohsome.org/v1/contributions/count/groupBy/boundary"

tags_de_interesse = {
    'leisure': '*',
    'building': '*',
    'amenity': '*'
}

params_contributions_base = {
    'time': '2007-10-08/2024-03-10',
    'contributionType': 'tagChange'
}

def process_response(response, cell_id):
    data = response.json()
    return sum(
        result.get('value', 0)
        for result in data.get('groupByResult', [])[0].get('result', [])
    )


for lot_id, subset in enumerate(grid_subset2, start=1):
    for feature in subset['features']:
        cell_geojson = json.dumps({
            "type": "FeatureCollection",
            "features": [feature]
        })
        cell_id = feature['properties']['id']

        for tag in tags_de_interesse:
            params_contributions = params_contributions_base.copy()
            params_contributions.update({
                'bpolys': cell_geojson,
                'filter': f'{tag}=* and name=*'
            })

            try:
                response = post_with_retry(
                    url_contributions,
                    params_contributions
                )
                contributions_count = process_response(response, cell_id)
            except RuntimeError:
                print(
                    f"Skipping cell {cell_id} for tag {tag} "
                    f"(connection failed)"
                )
                contributions_count = 0

            feature['properties'][f'{tag}_name_tagChange_contributions'] = contributions_count

    print(f"{subset['lot_id']} successfully processed!")

# Stop the time counter
end_time = time.time()

lot1 successfully processed!


In [ ]:
# Save or append step results to a GeoJSON file

# Create a FeatureCollection from the current step
current_step_fc = {
    'type': 'FeatureCollection',
    'crs': grid_subset2[0]['crs'],
    'features': []
}

for subset in grid_subset2:
    current_step_fc['features'].extend(subset['features'])

# Check if file already exists
if os.path.exists(output_filename):
    # Load existing features
    with open(output_filename, 'r', encoding='utf-8') as f:
        existing_data = json.load(f)
    # Append current step features
    existing_data['features'].extend(current_step_fc['features'])
    combined_fc = existing_data
else:
    combined_fc = current_step_fc

# Save combined results back to the file
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(combined_fc, f, ensure_ascii=False, indent=2)

print(f"Step results appended and saved to: {output_filename}")

#### Step 5 (API Endpoint: Users Aggregation): Count the number of users (contributors) who edited features with attribute name filled in:

- Count the number of users who edited features of the OSM tags of Interest with attribute "name" attribute filled in, aggregated by grid cells.

- Period of data retrieved: 2007-10-08 to 2024-03-10.


In [ ]:
# Step 5 (API Endpoint: Users Aggregation)
# Count the number of users (contributors)
# who edited features with attribute "name" filled in
# POST /users/count/groupBy/boundary

start_time = time.time()

url_users_count = "https://api.ohsome.org/v1/users/count/groupBy/boundary"

tags_de_interesse = {
    'leisure': '*',
    'building': '*',
    'amenity': '*'
}

params_users_count_base = {
    'time': '2007-10-08/2024-03-10'
}

def process_user_response(response, cell_id):
    data = response.json()
    for result in data.get('groupByResult', []):
        if result.get('groupByObject') == cell_id:
            return result.get('result', [{}])[0].get('value', 0)
    return 0


for lot_id, subset in enumerate(grid_subset2, start=1):
    for feature in subset['features']:
        cell_geojson = json.dumps({
            "type": "FeatureCollection",
            "features": [feature]
        })
        cell_id = feature['properties']['id']

        for tag in tags_de_interesse:
            params_users_count = params_users_count_base.copy()
            params_users_count.update({
                'bpolys': cell_geojson,
                'filter': f'{tag}=* and name=*'
            })

            try:
                response = post_with_retry(
                    url_users_count,
                    params_users_count
                )
                users_name_count = process_user_response(response, cell_id)
            except RuntimeError:
                print(
                    f"Skipping cell {cell_id} for tag {tag} "
                    f"(connection failed)"
                )
                users_name_count = 0

            feature['properties'][
                f'{tag}_users_count_name'
            ] = users_name_count

    print(f"{subset['lot_id']} successfully processed!")

end_time = time.time()
total_time_seconds = end_time - start_time
print(
    f"Total execution time: {total_time_seconds // 60} minutes "
    f"and {total_time_seconds % 60} seconds"
)

#### Save the updated grid cells with the information Extracted using the OHSOME API endpoints

In [ ]:
# Save or append step results to a GeoJSON file

# Create a FeatureCollection from the current step
current_step_fc = {
    'type': 'FeatureCollection',
    'crs': grid_subset2[0]['crs'],
    'features': []
}

for subset in grid_subset2:
    current_step_fc['features'].extend(subset['features'])

# Check if file already exists
if os.path.exists(output_filename):
    # Load existing features
    with open(output_filename, 'r', encoding='utf-8') as f:
        existing_data = json.load(f)
    # Append current step features
    existing_data['features'].extend(current_step_fc['features'])
    combined_fc = existing_data
else:
    combined_fc = current_step_fc

# Save combined results back to the file
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(combined_fc, f, ensure_ascii=False, indent=2)

print(f"Step results appended and saved to: {output_filename}")

---